In [64]:
%load_ext autoreload
%autoreload 2

INFO:root:Initialized contextual BERT-based model with name cseb_senticoref_suk.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
from main import read_file
import classla
classla.download("sl", dir="../models/classla_resources")

2023-05-25 20:36:20 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

INFO:classla:Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2023-05-25 20:36:31 INFO: File exists: ../models/classla_resources/sl/pos/standard.pt.
INFO:classla:File exists: ../models/classla_resources/sl/pos/standard.pt.
2023-05-25 20:36:31 INFO: File exists: ../models/classla_resources/sl/lemma/standard.pt.
INFO:classla:File exists: ../models/classla_resources/sl/lemma/standard.pt.
2023-05-25 20:36:32 INFO: File exists: ../models/classla_resources/sl/depparse/standard.pt.
I

In [66]:
preprocess = classla.Pipeline("sl", dir="../models/classla_resources", processors="tokenize,pos,lemma,ner")

2023-05-25 20:36:35 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| ner       | standard |

INFO:classla:Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| ner       | standard |

2023-05-25 20:36:35 INFO: Use device: cpu
INFO:classla:Use device: cpu
2023-05-25 20:36:35 INFO: Loading: tokenize
INFO:classla:Loading: tokenize
2023-05-25 20:36:35 INFO: Loading: pos
INFO:classla:Loading: pos
2023-05-25 20:36:59 INFO: Loading: lemma
INFO:classla:Loading: lemma
2023-05-25 20:37:32 INFO: Loading: ner
INFO:classla:Loading: ner
2023-05-25 20:37:32 INFO: Done loading processors!
INFO:classla:Done loading processors!


In [67]:
from collections import Counter

def get_named_entities(classla_doc, noun_ch_list, noun_thresh_perc=0.5, noun_ne_thresh_perc=0.1):
    named_entities_in_doc = []
    nouns_in_doc = []
    for sentence in classla_doc.sentences:
        mention_tokens = []
        for token in sentence.tokens:
            word = token.words[0]
            if len(mention_tokens) > 0 and not token.ner.endswith("-PER"):
                key = " ".join([t.words[0].lemma.lower() for t in mention_tokens])
                named_entities_in_doc.append(key)
                mention_tokens = []
            if token.ner.endswith('-PER'):
                mention_tokens.append(token)
            elif word.xpos[0] == "N":
                nouns_in_doc.append(word.lemma.lower())
        if len(mention_tokens) > 0:
            named_entities_in_doc.append(" ".join([t.words[0].lemma.lower() for t in mention_tokens]))
    nelf = dict(Counter(named_entities_in_doc))
    nlf = dict(Counter(nouns_in_doc))
    nlf_lim = max(nlf.values()) * noun_thresh_perc
    nlf_n_lim = max(nlf.values()) * noun_ne_thresh_perc
    nlf = {k: v for k, v in nlf.items() if v > nlf_lim or k.lower() in noun_ch_list and v > nlf_n_lim}
    return nelf, nlf

def find_mention(id, mentions):
    for mention in mentions:
        if id == mention.mention_id:
            return mention
    return None

def get_relevant_mentions(corefs, ments, ne_candidates):
    relevant_corefs = {k: vals for k, vals in corefs.items() if " ".join([t.lemma.lower() for t in find_mention(k, ments).tokens]) in ne_candidates}
    relevant_mention_ids = set()
    for k, vals in relevant_corefs.items():
        relevant_mention_ids.add(k)
        relevant_mention_ids.update(vals)
    relevant_mentions = [m for m in ments if m.mention_id in relevant_mention_ids]
    return relevant_corefs, relevant_mentions

In [68]:
from coref.resolve_text import Resolver

resolver = Resolver()

INFO:root:Initialized contextual BERT-based model with name cseb_senticoref_suk.


In [109]:
def merge_lemma_counts(lemma_counts):
    ret = {}
    lc_keys = list(lemma_counts.keys())
    used_keys = {k: False for k in lc_keys}
    k_len = len(lc_keys)
    for i in range(k_len - 1):
        for j in range(i + 1, k_len):
            lemma_i_tokens = lc_keys[i].split(" ")
            lemma_j_tokens = lc_keys[j].split(" ")
            if len(list(set(lemma_i_tokens) & set(lemma_j_tokens))) > 0:
                max_k = lc_keys[i] if len(lemma_i_tokens) >= len(lemma_j_tokens) else lc_keys[j]
                min_k = lc_keys[j] if len(lemma_i_tokens) >= len(lemma_j_tokens) else lc_keys[i]
                used_keys[max_k] = True
                used_keys[min_k] = True
                if max_k not in ret:
                    ret[max_k] = lemma_counts[max_k]
                ret[max_k] += lemma_counts[min_k]
    non_used_keys = [k for k, v in used_keys.items() if not v]
    for k in non_used_keys:
        ret[k] = lemma_counts[k]
    return ret

def merge_coreferences(coreferences, mentions):
    ret = {}
    cr_keys = list(coreferences.keys())
    used_keys = {k: False for k in cr_keys}
    k_len = len(cr_keys)
    for i in range(k_len - 1):
        for j in range(i + 1, k_len):
            coref_i_tokens = [t.lemma.lower() for t in find_mention(cr_keys[i], mentions).tokens]
            coref_j_tokens = [t.lemma.lower() for t in find_mention(cr_keys[j], mentions).tokens]
            if len(list(set(coref_i_tokens) & set(coref_j_tokens))) > 0:
                max_k = cr_keys[i] if len(coref_i_tokens) >= len(coref_j_tokens) else cr_keys[j]
                min_k = cr_keys[j] if len(coref_i_tokens) >= len(coref_j_tokens) else cr_keys[i]
                used_keys[max_k] = True
                used_keys[min_k] = True
                if max_k not in ret:
                    ret[max_k] = coreferences[max_k]
                ret[max_k].update(coreferences[min_k])
                ret[max_k].add(min_k)
    non_used_keys = [k for k, v in used_keys.items() if not v]
    for k in non_used_keys:
        ret[k] = coreferences[k]
    return ret

def get_locations(coreferences, rmentions, nmentions):
    locations = {}
    used_ids = []
    for k, v in coreferences.items():
        mention = find_mention(k, rmentions)
        used_ids.append(mention.mention_id)
        key = " ".join([t.lemma.lower() for t in mention.tokens])
        if key not in locations:
            locations[key] = set()
        locations[key].add((mention.tokens[0].sentence_index, mention.tokens[0].start_char, mention.tokens[0].start_char + len(key)))
        for coref in v:
            m = find_mention(coref, rmentions)
            used_ids.append(m.mention_id)
            locations[key].add((mention.tokens[0].sentence_index, m.tokens[0].start_char, m.tokens[0].start_char + len(m.tokens[0].raw_text)))
    
    for mention in nmentions:
        if mention.mention_id not in used_ids:
            key = " ".join([t.lemma.lower() for t in mention.tokens])
            if key not in locations:
                locations[key] = set()
            locations[key].add((mention.tokens[0].sentence_index, mention.tokens[0].start_char, mention.tokens[0].start_char + len(key)))
    return locations
                
def get_counts(coreferences, rmentions, nmentions):
    counts = {}
    used_ids = []
    for k, v in coreferences.items():
        mention = find_mention(k, rmentions)
        used_ids.append(mention.mention_id)
        key = " ".join([t.lemma.lower() for t in mention.tokens])
        if key not in counts:
            counts[key] = 0
        counts[key] += 1 + len(v)
    
    for mention in nmentions:
        if mention.mention_id not in used_ids:
            key = " ".join([t.lemma.lower() for t in mention.tokens])
            if key not in counts:
                counts[key] = 0
            counts[key] += 1
    return counts

def get_most_mentioned_characters(text, ne_nouns):
    doc = preprocess(text)
    _1, _2 = get_named_entities(doc, ne_nouns)
    ne_candidates = {**_1, **_2}
    coref_output = resolver.coref(doc, ne_candidates, 0.6, 20, 15)
    mentions, coreferences = coref_output
    noun_mentions = [m for m in mentions if m.tokens[0].msd[0] == "N"]
    relevant_coreferences, relevant_mentions = get_relevant_mentions(coreferences, mentions, ne_candidates)
    lemma_counts = get_counts(relevant_coreferences, relevant_mentions, noun_mentions)
    lemma_counts = merge_lemma_counts(lemma_counts)
    relevant_coreferences = merge_coreferences(relevant_coreferences, relevant_mentions)
    locations = get_locations(relevant_coreferences, relevant_mentions, noun_mentions)
    return lemma_counts, locations
    

In [95]:
# text = read_file("../data/slovenian_short_stories/Čudno_drevo.txt")
# with open("../models/slo_ne.txt") as f:
#     ne_nouns = [line.strip() for line in f]
# lemma_counts, locations = get_most_mentioned_characters(text, ne_nouns)

INFO:root:Evaluating a single document...


0:20


INFO:root:Evaluating a single document...


15:35


INFO:root:Evaluating a single document...


...

90:110


INFO:root:Evaluating a single document...


105:125


INFO:root:Evaluating a single document...


120:133


In [116]:
import os

with open("../models/slo_ne.txt") as f:
    ne_nouns = [line.strip() for line in f]

counts = []
locations = []
for f in os.listdir('../data/slovenian_novels/'):
    raw_text = read_file(f'../data/slovenian_novels/{f}')
    _counts, _locations = get_most_mentioned_characters(raw_text, ne_nouns)
    counts.append(_counts)
    locations.append(_locations)


INFO:root:Evaluating a single document...


0:20


INFO:root:Evaluating a single document...


15:35


INFO:root:Evaluating a single document...


...

360:380


INFO:root:Evaluating a single document...


375:395


INFO:root:Evaluating a single document...


390:406


In [119]:
locations[0]

{'zemlja': {(23, 2544, 2550),
  (37, 4144, 4150),
  (37, 4726, 4732),
  (37, 7645, 7651),
  (37, 7659, 7661),
  (43, 4979, 4985),
  (44, 5143, 5149),
  (54, 5996, 6002)},
 'marija': {(18, 2395, 2401), (18, 2406, 2410), (102, 10165, 10171)},
 'oči': {(2, 361, 364),
  (13, 1798, 1801),
  (15, 2095, 2098),
  (54, 6073, 6076),
  (89, 8698, 8701),
  (90, 8900, 8903),
  (90, 8979, 8982),
  (94, 9350, 9353),
  (109, 11110, 11113),
  (111, 11337, 11340),
  (112, 11573, 11576)},
 'glas': {(9, 1181, 1185),
  (11, 1499, 1503),
  (23, 2519, 2523),
  (40, 4630, 4634),
  (52, 5867, 5871),
  (57, 6312, 6316),
  (96, 9577, 9581),
  (102, 10215, 10219)},
 'dunajčan': {(12, 1598, 1606), (54, 5978, 5986), (59, 6380, 6388)},
 'servus': {(13, 1863, 1869)},
 'korak': {(15, 2025, 2030),
  (25, 2724, 2729),
  (26, 2913, 2918),
  (31, 3429, 3434),
  (34, 3777, 3782),
  (36, 3977, 3982),
  (36, 3986, 3991),
  (37, 4206, 4211),
  (40, 4554, 4559),
  (64, 6770, 6775),
  (87, 8457, 8462),
  (114, 11678, 11683)},
 

In [120]:
import json

def write_output(out_list, fname):
    json_data = json.dumps(out_list, indent=4)
    with open(f"../data/{fname}.json", "w") as f:
        f.write(json_data)

write_output(counts, "characters_by_stories_slo_novels")

In [121]:
def write_locations(locations, fname):
    out_locations = []
    for loc in locations:
        out_locations.append({k: list(map(lambda l: {"sentence_index": l[0], "start_idx": l[1], "end_idx": l[2]}, v)) for k, v in loc.items()})
    json_data = json.dumps(out_locations, indent=4)
    with open(f"../data/{fname}.json", "w") as f:
        f.write(json_data)

write_locations(locations, "named_entities_by_stories_slo_novels")